In [1]:
import cv2
import os
from tqdm import tqdm

In [2]:
from pathlib import Path
from PIL import Image

In [3]:
BROAD_TRACK_DIR = Path(r"C:\Users\ABHINAV\Desktop\player-reid\broadcast_labels")  
TACTI_TRACK_DIR = Path(r"C:\Users\ABHINAV\Desktop\player-reid\tacticam_labels")
OUTPUT_DIR = Path(r"C:\Users\ABHINAV\Desktop\player-reid\Output")       
# Create output folder if it doesn’t exist
#OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print("Broadcast Track source:", BROAD_TRACK_DIR)
print("Tacticam Track source:", TACTI_TRACK_DIR)
print("Saving crops to:", OUTPUT_DIR)

Broadcast Track source: C:\Users\ABHINAV\Desktop\player-reid\broadcast_labels
Tacticam Track source: C:\Users\ABHINAV\Desktop\player-reid\tacticam_labels
Saving crops to: C:\Users\ABHINAV\Desktop\player-reid\Output


def extract_crops_from_labels(label_folder, video_path, output_root):
    cap = cv2.VideoCapture(str(video_path))
    frame_cache = {}

    print(f"Opening video: {video_path}")
    cap = cv2.VideoCapture(str(video_path))
    ret, test_frame = cap.read()
    print("Frame loaded:", ret, "| Frame shape:" if ret else "", test_frame.shape if ret else "N/A")
    cap.release()


    for label_file in os.listdir(label_folder):
        if not label_file.endswith(".txt"):
            continue

        with open(os.path.join(label_folder, label_file), "r") as f:
            lines = f.readlines()

        for line in lines:
            parts = line.strip().split()
            if len(parts) < 6:
                continue  # skip malformed lines
            frame_id = int(parts[0])
            track_id = parts[1]
            x1, y1, x2, y2 = map(int, map(float, parts[2:6]))

            # Get frame
            if frame_id not in frame_cache:
                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
                ret, frame = cap.read()
                if not ret:
                    continue
                frame_cache[frame_id] = frame
            else:
                frame = frame_cache[frame_id]

            crop = frame[int(y1):int(y2), int(x1):int(x2)]

            # Validate crop
            if crop is None or crop.size == 0:
                print(f"⚠️ Empty crop at frame {frame_id} for track {track_id} in {label_file}")
                continue

            cam_name = os.path.splitext(label_file)[0]
            track_folder = Path(output_root) / cam_name / track_id
            track_folder.mkdir(parents=True, exist_ok=True)

            crop_filename = track_folder / f"{frame_id}.jpg"
            cv2.imwrite(str(crop_filename), crop)

    cap.release()

# === SETUP WITH YOUR CURRENT FOLDER STRUCTURE ===
label_folder = Path(r"C:\Users\ABHINAV\Desktop\player-reid\botsort_broadcast\labels")         
video_path = Path(r"C:\Users\ABHINAV\Desktop\player-reid\broadcast.avi")     
output_root = Path(r"C:\Users\ABHINAV\Desktop\player-reid\Output")

# === RUN ===
extract_crops_from_labels(label_folder, video_path, output_root)


#file_path = r"C:\Users\ABHINAV\Desktop\player-reid\botsort_tacticam\labels\tacticam_1.txt"
# tacticam_2.txt, tacticam_3.txt, etc.
file_path = r"C:\Users\ABHINAV\Desktop\player-reid\botsort_tacticam\labels\tacticam_2.txt"

with open(file_path, "r") as f:
    lines = f.readlines()
    for i, line in enumerate(lines[:5]):
        print(f"Sample line {i+1}:", line.strip())



BROAD_TRACK_DIR = Path(r"C:\Users\ABHINAV\Desktop\player-reid\botsort_broadcast\labels")
BROAD_VIDEO_PATH = Path(r"C:\Users\ABHINAV\Desktop\player-reid\broadcast.mp4")
BROAD_OUTPUT_DIR = Path(r"C:\Users\ABHINAV\Desktop\player-reid\Output\broadcast")

TACTI_TRACK_DIR = Path(r"C:\Users\ABHINAV\Desktop\player-reid\botsort_tacticam\labels")
TACTI_VIDEO_PATH = Path(r"C:\Users\ABHINAV\Desktop\player-reid\tacticam.mp4")
TACTI_OUTPUT_DIR = Path(r"C:\Users\ABHINAV\Desktop\player-reid\Output\tacticam")


import cv2
from pathlib import Path
import os
from tqdm import tqdm

def extract_crops_from_labels(label_folder, video_path, output_root):
    label_files = sorted(label_folder.glob("*.txt"))
    print(f"Label files found: {len(label_files)}")

    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        print("❌ Failed to open video:", video_path)
        return

    frame_cache = {}  # Cache to avoid reading same frame multiple times

    for label_file in tqdm(label_files, desc="Processing labels"):
        with open(label_file, "r") as f:
            lines = f.readlines()

        for line in lines:
            parts = line.strip().split()
            if len(parts) != 6:
                continue  # skip malformed lines

            frame_id = int(parts[0])
            x_center, y_center, w, h = map(float, parts[1:5])
            track_id = parts[5]

            # Read frame if not cached
            if frame_id not in frame_cache:
                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
                success, frame = cap.read()
                if not success:
                    print(f"⚠️ Failed to read frame {frame_id}")
                    continue
                frame_cache[frame_id] = frame
            else:
                frame = frame_cache[frame_id]

            H, W = frame.shape[:2]

            # Convert normalized YOLO box → pixel box
            x_center *= W
            y_center *= H
            w *= W
            h *= H

            x1 = int(x_center - w / 2)
            y1 = int(y_center - h / 2)
            x2 = int(x_center + w / 2)
            y2 = int(y_center + h / 2)

            # Clip to frame bounds
            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(W - 1, x2), min(H - 1, y2)

            crop = frame[y1:y2, x1:x2]
            if crop.size == 0:
                continue

            track_folder = output_root / f"id_{track_id}"
            track_folder.mkdir(parents=True, exist_ok=True)

            crop_filename = track_folder / f"{frame_id}.jpg"
            cv2.imwrite(str(crop_filename), crop)

    cap.release()
    print("✅ Done extracting crops.")


extract_crops_from_labels(BROAD_TRACK_DIR, BROAD_VIDEO_PATH, BROAD_OUTPUT_DIR)
extract_crops_from_labels(TACTI_TRACK_DIR, TACTI_VIDEO_PATH, TACTI_OUTPUT_DIR)

In [5]:
import cv2
from pathlib import Path
import os
from tqdm import tqdm

def extract_crops_from_labels(label_folder, video_path, output_root):
    label_files = sorted(label_folder.glob("*.txt"))
    print(f"Label files found in {label_folder}: {len(label_files)}")

    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        print("❌ Failed to open video:", video_path)
        return

    frame_cache = {}

    for label_file in tqdm(label_files, desc=f"Processing {video_path.name} labels"):
        with open(label_file, "r") as f:
            lines = f.readlines()

        for line in lines:
            parts = line.strip().split()
            if len(parts) != 6:
                continue  # Skip malformed lines

            frame_id = int(parts[0])
            x_center, y_center, w, h = map(float, parts[1:5])
            track_id = parts[5]

            if frame_id not in frame_cache:
                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
                success, frame = cap.read()
                if not success:
                    print(f"⚠️ Failed to read frame {frame_id}")
                    continue
                frame_cache[frame_id] = frame
            else:
                frame = frame_cache[frame_id]

            H, W = frame.shape[:2]

            # Convert normalized YOLO box → pixel box
            x_center *= W
            y_center *= H
            w *= W
            h *= H

            x1 = int(x_center - w / 2)
            y1 = int(y_center - h / 2)
            x2 = int(x_center + w / 2)
            y2 = int(y_center + h / 2)

            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(W - 1, x2), min(H - 1, y2)

            crop = frame[y1:y2, x1:x2]
            if crop.size == 0:
                continue

            track_folder = output_root / f"id_{track_id}"
            track_folder.mkdir(parents=True, exist_ok=True)

            crop_filename = track_folder / f"{frame_id}.jpg"
            cv2.imwrite(str(crop_filename), crop)

    cap.release()
    print(f"✅ Done extracting crops to {output_root}")


# === Updated paths from user ===
BROAD_TRACK_DIR = Path(r"C:\Users\ABHINAV\Desktop\player-reid\broadcast_labels")
TACTI_TRACK_DIR = Path(r"C:\Users\ABHINAV\Desktop\player-reid\tacticam_labels")
OUTPUT_DIR = Path(r"C:\Users\ABHINAV\Desktop\player-reid\Output")

BROAD_VIDEO_PATH = Path(r"C:\Users\ABHINAV\Desktop\player-reid\broadcast.mp4")
TACTI_VIDEO_PATH = Path(r"C:\Users\ABHINAV\Desktop\player-reid\tacticam.mp4")

# Print confirmation
print("Broadcast Track source:", BROAD_TRACK_DIR)
print("Tacticam Track source:", TACTI_TRACK_DIR)
print("Saving crops to:", OUTPUT_DIR)

# === Run both ===
extract_crops_from_labels(BROAD_TRACK_DIR, BROAD_VIDEO_PATH, OUTPUT_DIR / "broadcast")
extract_crops_from_labels(TACTI_TRACK_DIR, TACTI_VIDEO_PATH, OUTPUT_DIR / "tacticam")


Broadcast Track source: C:\Users\ABHINAV\Desktop\player-reid\broadcast_labels
Tacticam Track source: C:\Users\ABHINAV\Desktop\player-reid\tacticam_labels
Saving crops to: C:\Users\ABHINAV\Desktop\player-reid\Output
Label files found in C:\Users\ABHINAV\Desktop\player-reid\broadcast_labels: 0


Processing broadcast.mp4 labels: 0it [00:00, ?it/s]


✅ Done extracting crops to C:\Users\ABHINAV\Desktop\player-reid\Output\broadcast
Label files found in C:\Users\ABHINAV\Desktop\player-reid\tacticam_labels: 0


Processing tacticam.mp4 labels: 0it [00:00, ?it/s]

✅ Done extracting crops to C:\Users\ABHINAV\Desktop\player-reid\Output\tacticam


In [ ]:
extract_crops_from_yolo_labels(
    frames_dir=r"C:\Users\ABHINAV\Desktop\player-reid\Frames\broadcast",
    labels_dir=r"C:\Users\ABHINAV\Desktop\player-reid\Labels\broadcast",
    output_dir=r"C:\Users\ABHINAV\Desktop\player-reid\Output\broadcast"
)